
# FoodBot Recipe (Prolog Code)

In [ ]:
!apt install swi-prolog
!pip install pyswip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swi-prolog is already the newest version (7.6.4+dfsg-1build1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
from pyswip import Prolog
import requests
import json

### Detail on what your expert system will focus upon, and what the askables will be. You need to have a minimum of 8 askables in your expert system, regardless of the domain of your KB. [#rightproblem]

We will be using the expert system example provided in the instructions, of helping students find suitable recipes so they are incentivized to eat inside and lower risk of infection. The expert system will function on the following 8 askables: "How much time does the student have to make the meal?", "How much effort in cooking do you want?", "What type of meal do you want? (Appetizer, Main, Dessert)" "Do you have any dietary goals? (Pleasure, Weight Loss, Weight Gain, etc...", "Which cuisine do you prefer?", "Which diet do you prefer?", "Ingredients to look for?", "How many calories are you aiming for?". Some of these askables are dependent on each other. For example, if the person has a specific diet, that will influence the available options for asking what their preferred cuisine is because we may not have their diet available in all cusines.

### Perform data collection for building your expert system by surveying resources available online, guided by the askables that you have chosen. [\#evidencebased, \#sourcequality].
The data will be collected from the following scraped data: https://raw.githubusercontent.com/mneedham/bbcgoodfood/master/stream_all.json. It is from a github project that scraped BBC's Good Food website for information on their recipes. While it has thousands of data points, the data needed cleaning, so the first 100 were cleaned and kept in a new JSON file presented here: https://gist.githubusercontent.com/pierre-minerva/70bccece0820fa839b53c264cc7460a3/raw/8c7afccd58cf78b42b99ceb0927025896ea108fa/recipe_data.json

In [ ]:
recipes = []

#get data
r = requests.get("https://gist.githubusercontent.com/pierre-minerva/70bccece0820fa839b53c264cc7460a3/raw/8c7afccd58cf78b42b99ceb0927025896ea108fa/recipe_data.json")
#Save db
with open('recipes.json', 'wb') as f:
    f.write(r.content)

#process data
with open('recipes.json') as f:
    for jsonObj in f:
        recipesDict = json.loads(jsonObj)
        recipes.append(recipesDict)

### Explain the logic of your expert system by showing which values of the askables lead to what specific information being provided to the end user. Any visualization that makes the logic of your expert system clear is acceptable (for example, a tree diagram or table). [#ailogic]

  The logic is very straightforward. 
1. First, we ask if they follow a specific diet (i.e. low-carb) in multiple choice format based on the literals from the data collection. This simply filters out all possible recipes that do not match the user preference. 
2. Then we ask if they have a dietary goal. The options are "Weight-Loss", "Weight-Gain", "Nutrition", and "Pleasure". 
3. "Weight-Loss" and "Weight-Gain" prompt a second question asking for the user's target caloric intake, which is used to filter out more results. "Nutrition" filters for recipes with 20g of protein or more. "Pleasure" doesn't filter for anything. 
4. Then we ask about total time available to prepare the meal, with options between <5m, <15m, <30m, >30m, filtering appropriately. 
5. Then we ask for the desired involvement in the cooking process, with the following choices "Easy", "Some Effort", and "A Challenge". 
6. Then, we ask for the type of meal, the user wants from "Snack", "Main Course", "Dessert", and "Breakfast". If they selected "Weight-Loss" or "Nutrion" earlier, the "Dessert" option will be left out. If "Weight-Gain", "Snack" is left out. 
7. Then we ask if the user has any ingredients they would like to use from the remaining recipes that still match the previously selected questions. 
8. Then we ask if the user has any preferred cusine from the recipes remaining that still match the previously selected questsions.

### Using this visualization to help you, code your expert system using either a native Prolog front-end, or one using the PySWIP library to interface a Python frontend. [#aicoding]

In [ ]:
import re

def list_to_dict(lst):
  d = {}
  l = list(set(lst))

  for i in range(len(l)):
    d[i] = l[i]
  
  return d, l

def main():
  prolog = Prolog()

  #Adding of data to KB
  for recipe in recipes:
    #recipe
    title = recipe['page']['title']
    #diet
    for diet in recipe['page']['recipe']['diet_types']:
      prolog.assertz(f"diet('{title}','{diet}')")
    #protein
    protein = int(re.search(r'\d+', recipes[0]['page']['recipe']['nutrition_info'][2]).group())
    prolog.assertz(f"protein('{title}',{protein})")
    #calories
    calories = int(re.search(r'\d+', recipes[0]['page']['recipe']['nutrition_info'][1]).group())
    prolog.assertz(f"calories('{title}',{calories})")
    #time
    total_time = (recipe['page']['recipe']['cooking_time'] + recipes[0]['page']['recipe']['prep_time'])//60
    prolog.assertz(f"time('{title}',{total_time})")
    #effort
    skill_level = recipe['page']['recipe']['skill_level']
    prolog.assertz(f"effort('{title}','{skill_level}')")
    #type
    for course in recipe['page']['recipe']['courses']:
      prolog.assertz(f"course('{title}','{course}')")
    #ingredients
    for ing in recipe['page']['recipe']['ingredients']:
      prolog.assertz(f"ing('{title}','{ing}')")
    #cuisine
    cuisine = recipe['page']['recipe']['cusine']
    prolog.assertz(f"cuisine('{title}','{cuisine}')")
    #recipe
    prolog.assertz(f"recipe('{title}','{diet}',{protein},{calories},{total_time},'{skill_level}','{course}','{ing}','{cuisine}' )")

  prolog.assertz("more(X,Y) :- X @> Y")

  #Q1
  prolog.assertz("diet(D)")

  #Q2
  prolog.assertz("goal(G)")

  #Q3
  prolog.assertz("protein(R) :- goal('Nutrition'); protein(R,P); more(P,20)")

  #Q6
  #if loss then no dessert

  prolog.assertz("recipe2(R,D,Ci,Ti,E,Co,I,Cu)")
  prolog.assertz("recipe2(R,D,Ci,Ti,E,Co,I,Cu) :- recipe(R,D,P,C,T,E,Co,I,Cu); more(T,Ti); goal('Weight-Loss'); more(Ci,C); dif(Co,'Dessert')")
  prolog.assertz("recipe2(R,D,Ci,Ti,E,Co,I,Cu) :- recipe(R,D,P,C,T,E,Co,I,Cu); more(T,Ti); goal('Weight-Gain'); more(C,Ci); dif(Co,'Snack')")
  prolog.assertz("recipe2(R,D,Ci,Ti,E,Co,I,Cu) :- recipe(R,D,P,C,T,E,Co,I,Cu); more(T,Ti); goal('Nutrition'); more(P,20); dif(Co,'Dessert')")
  prolog.assertz("recipe2(R,D,Ci,Ti,E,Co,I,Cu) :- recipe(R,D,P,C,T,E,Co,I,Cu); more(T,Ti); goal('Pleasure')")

  #Q1
  diets = []

  for soln in prolog.query("diet(R,D)"):
    if "_" not in str(soln["D"]):
      diets.append(str(soln["D"]))

  diets_dict, diets_lst = list_to_dict(diets)
  print("Do you have any dietary preferences?")
  print("Input number of choice:")
  print(diets_dict)
  diet_choice = int(input())
  prolog.assertz(f"diet('{diets_dict[diet_choice]}')")

  #Q2
  goal_dict = {0:"Weight-Loss", 1: "Weight-Gain", 2:"Nutrition", 3:"Pleasure"}
  print("What is your dietary goal?")
  print("Input number of choice:")
  print(goal_dict)
  goal_choice = int(input())
  prolog.assertz(f"goal('{goal_dict[goal_choice]}')")

  #Q3
  if goal_choice == 0 or goal_choice == 1:
    print("What are your target calories?")
    calories_choice = int(input())
    prolog.assertz(f"calories({calories_choice})")
  else:
    calories_choice = 9999
    prolog.assertz(f"calories({calories_choice})")

  #Q4
  print("How many minutes do you have to make the food?")
  time_choice = int(input())
  prolog.assertz(f"time({time_choice})")

  #Q5
  effort = []

  for soln in prolog.query(f"recipe2(R,'{diets_dict[diet_choice]}',{calories_choice},{time_choice},E,Co,I, Cu)"):
    if "_" not in str(soln["E"]):
      effort.append(str(soln["E"]))

  effort_dict, effort_lst = list_to_dict(effort)
  print("How much effort do you want to put in?")
  print("Input number of choice:")
  print(effort_dict)
  effort_choice = int(input())
  prolog.assertz(f"effort('{effort_dict[effort_choice]}')")

  #Q6
  course = []

  for soln in prolog.query(f"recipe2(R,'{diets_dict[diet_choice]}',{calories_choice},{time_choice},'{effort_dict[effort_choice]}',Co,I, Cu)"):
    if str(soln["Co"]) in ["Snack", "Main course", "Dessert", "Breakfast"]:
      course.append(str(soln["Co"]))

  course_dict, course_lst = list_to_dict(course)
  print("What course would you like?")
  print("Input number of choice:")
  print(course_dict)
  course_choice = int(input())
  prolog.assertz(f"course('{course_dict[course_choice]}')")

  #Q7
  ing = []

  for soln in prolog.query(f"recipe2(R,'{diets_dict[diet_choice]}',{calories_choice},{time_choice},'{effort_dict[effort_choice]}','{course_dict[course_choice]}',I, Cu)"):
    if "_" not in str(soln["I"]):
      ing.append(str(soln["I"]))

  ing_dict, ing_lst = list_to_dict(ing)
  print("Which of these remaining ingredients would you like to be in your meal?")
  print("Input number of choice:")
  print(ing_dict)
  ing_choice = int(input())
  prolog.assertz(f"ing('{ing_dict[ing_choice]}')")

  #Q8
  cuisine = []

  for soln in prolog.query(f"recipe2(R,'{diets_dict[diet_choice]}',{calories_choice},{time_choice},'{effort_dict[effort_choice]}','{course_dict[course_choice]}','{ing_dict[ing_choice]}', Cu)"):
    if "_" not in str(soln["Cu"]):
      cuisine.append(str(soln["Cu"]))

  cuisine_dict, cuisine_lst = list_to_dict(cuisine)
  print("What cuisine do you want?")
  print("Input number of choice:")
  print(cuisine_dict)
  cuisine_choice = int(input())

  final_recipes = []
  for soln in prolog.query(f"recipe2(R,'{diets_dict[diet_choice]}',{calories_choice},{time_choice},'{effort_dict[effort_choice]}','{course_dict[course_choice]}','{ing_dict[ing_choice]}', '{cuisine_dict[cuisine_choice]}')"):
    if type(soln["R"]) == type("str"):
      final_recipes.append(soln["R"])

  final_recipes_dict, final_recipes_lst = list_to_dict(final_recipes)
  print("The following recipes from BBC's Good Foods website match your needs.")
  print(final_recipes_lst)

In [ ]:
#Test case 1
main()

Do you have any dietary preferences?
Input number of choice:
{0: 'Low-salt', 1: 'Low-calorie', 2: 'Heart Healthy', 3: 'Healthy', 4: 'Low-fat', 5: 'Gluten-free', 6: 'Dairy-free', 7: 'Vegetarian', 8: 'Egg-free', 9: 'Vegan'}
4
What is your dietary goal?
Input number of choice:
{0: 'Weight-Loss', 1: 'Weight-Gain', 2: 'Nutrition', 3: 'Pleasure'}
0
What are your target calories?
500
How many minutes do you have to make the food?
60
How much effort do you want to put in?
Input number of choice:
{0: 'Easy'}
0
What course would you like?
Input number of choice:
{0: 'Breakfast', 1: 'Main course'}
0
Which of these remaining ingredients would you like to be in your meal?
Input number of choice:
{0: 'gravadlax'}
0
What cuisine do you want?
Input number of choice:
{0: 'Scandinavian'}
0
The following recipes from BBC's Good Foods website match your needs.
['Potato &amp; dill pancakes with gravadlax']


In [ ]:
#Test case 2
main()

Do you have any dietary preferences?
Input number of choice:
{0: 'Low-salt', 1: 'Low-calorie', 2: 'Heart Healthy', 3: 'Healthy', 4: 'Low-fat', 5: 'Gluten-free', 6: 'Dairy-free', 7: 'Vegetarian', 8: 'Egg-free', 9: 'Vegan'}
2
What is your dietary goal?
Input number of choice:
{0: 'Weight-Loss', 1: 'Weight-Gain', 2: 'Nutrition', 3: 'Pleasure'}
1
What are your target calories?
200
How many minutes do you have to make the food?
60
How much effort do you want to put in?
Input number of choice:
{0: 'Easy'}
0
What course would you like?
Input number of choice:
{0: 'Dessert'}
0
Which of these remaining ingredients would you like to be in your meal?
Input number of choice:
{0: 'sea salt'}
0
What cuisine do you want?
Input number of choice:
{0: ''}
0
The following recipes from BBC's Good Foods website match your needs.
['Chocolate, peanut butter &amp; pretzel cookie bars']


In [ ]:
#Test case 3
main()

Do you have any dietary preferences?
Input number of choice:
{0: 'Low-salt', 1: 'Low-calorie', 2: 'Heart Healthy', 3: 'Healthy', 4: 'Low-fat', 5: 'Gluten-free', 6: 'Dairy-free', 7: 'Vegetarian', 8: 'Egg-free', 9: 'Vegan'}
0
What is your dietary goal?
Input number of choice:
{0: 'Weight-Loss', 1: 'Weight-Gain', 2: 'Nutrition', 3: 'Pleasure'}
3
How many minutes do you have to make the food?
60
How much effort do you want to put in?
Input number of choice:
{0: 'Easy', 1: 'A challenge', 2: 'More effort'}
0
What course would you like?
Input number of choice:
{0: 'Snack', 1: 'Dessert'}
0
Which of these remaining ingredients would you like to be in your meal?
Input number of choice:
{0: 'dried cranberries', 1: 'lemon'}
1
What cuisine do you want?
Input number of choice:
{0: 'British'}
0
The following recipes from BBC's Good Foods website match your needs.
['Rhubarb &amp; vanilla jam']
